<a href="https://colab.research.google.com/github/Matthew0077/genai/blob/main/HW9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Reflection 的任務設計

#### 🌟 任務說明：社群媒體發文優化小幫手

**🎯 流程說明：**
1. 使用者輸入今天想分享的內容
2. `model_writer` 生成第一版社群發文（風格活潑、有趣、使用 emoji、第一人稱）
3. `model_reviewer` 檢查內容是否夠生活化、通順、有趣，並提供具體修改建議
4. `model_writer` 根據建議產出第二版
5. Gradio 呈現：三個欄位：第一版、建議、第二版

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
api_key = userdata.get('OpenAI')
os.environ['OPENAI_API_KEY']=api_key
provider = "openai"
model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.16.1 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 基本的設定

In [4]:
import aisuite as ai

In [5]:
provider_writer = "groq"
model_writer="llama3-70b-8192"

provider_reviewer = "groq"
model_reviewer = "llama3-70b-8192"

provider_reviewer = "openai"
model_reviewer = "gpt-4o"

標準回應函式

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 設定「作者」和「審查員」

In [30]:
system_writer = "你是一位專業的線上戀愛顧問（Matchmaker GPT），年齡設定 25–30 歲，擅長 Z 世代溝通節奏與女性心理。任務：根據使用者最新訊息，生成「思考筆記」，分析情緒線索、心理需求、可能的回覆方向。"
system_reviewer = "你是一位 Matchmaker GPT，依據「思考筆記」產出 3 種回覆（標準／幽默／深入），語氣直白真實、兼具幽默與同理，2–4 句內完成。請注意：在回覆中，不要自稱「數位生物」或其他科技相關詞，只用「我」或你的綽號即可。"

In [8]:
def reflect_post(prompt):
    # Step 1: Writer 初稿
    first_version = reply(system_writer, prompt,
                          provider=provider_writer,
                          model=model_writer
                          )

    # Step 2: Reviewer 給建議
    suggestion = reply(system_reviewer, first_version,
                       provider=provider_reviewer,
                       model=model_reviewer
                       )

    # Step 3: Writer 再寫一次（根據建議）
    second_prompt = f"這是我剛剛寫的貼文：\n{first_version}\n\n這是修改建議：\n{suggestion}\n\n請根據這些建議，幫我改得更生活化、更自然。請用台灣習慣的中文, 並且只要輸出改好的文章就可以了。"
    second_version = reply(system_writer, second_prompt,
                           provider=provider_writer,
                           model=model_reviewer
                           )

    return first_version, suggestion, second_version

In [9]:
import json, os

MEMORY_FILE = "memory_store.json"

def load_memory():
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_memory(mem):
    with open(MEMORY_FILE, "w", encoding="utf-8") as f:
        json.dump(mem, f, ensure_ascii=False, indent=2)

# 初始化
memory = load_memory()

In [29]:
COT_SYSTEM_PROMPT = """
你是一位專業的線上戀愛顧問（Matchmaker GPT），
年齡設定 25–30 歲，擅長 Z 世代溝通節奏與女性心理。
任務：根據使用者最新訊息，生成「思考筆記」，
分析情緒線索、心理需求、可能的回覆方向。
"""

GEN_SYSTEM_PROMPT = """
你是一位 Matchmaker GPT，
依據「思考筆記」產出 3 種回覆（標準／幽默／深入），
語氣直白真實、兼具幽默與同理，2–4 句內完成，
請注意：在回覆中，不要自稱「數位生物」或其他科技相關詞，只用「我」或你的綽號即可。
"""

In [23]:
from openai import OpenAI

client = OpenAI()

def think_stage(user_msg, history):
    prompt = COT_SYSTEM_PROMPT + "\n\n" + "\n".join(history) + f"\nUser: {user_msg}\nThoughts:"
    res = client.chat.completions.create(model="gpt-4o", messages=[{"role":"system","content":prompt}])
    return res.choices[0].message.content.strip()

def gen_stage(thoughts, user_msg):
    prompt = GEN_SYSTEM_PROMPT + "\n\n思考筆記：" + thoughts + f"\nUser: {user_msg}\nReply:"
    res = client.chat.completions.create(model="gpt-4o", messages=[{"role":"system","content":prompt}])
    return res.choices[0].message.content.strip()

In [24]:
def handle_message(person_id, user_msg):
    # 1. 讀取對象記憶
    user_mem = memory.get(person_id, {"history": [], "prefs": {}})
    history = user_mem["history"]

    # 2. CoT 思考
    thoughts = think_stage(user_msg, history)

    # 3. Generation
    reply = gen_stage(thoughts, user_msg)

    # 4. 更新記憶
    history.append(f"User: {user_msg}")
    history.append(f"Bot: {reply}")
    user_mem["history"] = history[-20:]  # 保留近 20 條
    memory[person_id] = user_mem
    save_memory(memory)

    return reply

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [25]:
!pip install gradio

In [26]:
import os, json

if not os.path.exists(MEMORY_FILE):
    with open(MEMORY_FILE, "w", encoding="utf-8") as f:
        json.dump({}, f, ensure_ascii=False, indent=2)

In [27]:
import gradio as gr

In [31]:
import traceback

def gradio_handler(person_id, user_msg):
    try:
        return handle_message(person_id, user_msg)
    except Exception as e:
        # 將完整錯誤堆疊訊息回傳到介面
        return traceback.format_exc()

with gr.Blocks() as demo:
    person = gr.Textbox(label="對象識別 (ID 或暱稱)")
    msg_in = gr.Textbox(label="輸入訊息")
    chat_out = gr.Textbox(label="AI 回覆", interactive=False)
    download_btn = gr.File(label="下載記憶檔", value="memory_store.json", interactive=False)
    btn = gr.Button("送出")
    btn.click(fn=gradio_handler, inputs=[person, msg_in], outputs=[chat_out])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f71607d80d53f2e83.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
